**Exercise 23.1:**
Perform a network projection of this bipartite network using simple weights. The unipartite projection should only contain nodes of type 1 (|V1| = 248). How dense is the projection?

In [3]:
import networkx as nx

G = nx.read_edgelist('23_1_data.txt', using=nx)

TypeError: read_edgelist() got an unexpected keyword argument 'using'